In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("../../")

In [3]:
from dask.distributed import Client
client = Client()
client

2023-11-13 17:51:02,097 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-pqs5cjy4', purging
2023-11-13 17:51:02,098 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-bht_zuoc', purging
2023-11-13 17:51:02,098 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-6mwtpz6q', purging
2023-11-13 17:51:02,098 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-46l8_bqi', purging


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 16,Total memory: 12.44 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:45251,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 16
Started: Just now,Total memory: 12.44 GiB
Comm: tcp://127.0.0.1:33011,Total threads: 4
Dashboard: http://127.0.0.1:44837/status,Memory: 3.11 GiB
Nanny: tcp://127.0.0.1:38125,


In [4]:
import pipit as pp

In [5]:
trace = pp.Trace.from_otf2("/home/rakrish/otf2-mpi-only-traces/kripke/32/")

In [6]:
df = trace.events
df

,Timestamp (ns),Event Type,Name,Thread,Process,Attributes
npartitions=1,,,,,,
0,float64,category[known],category[known],category[known],category[known],object
11854,...,...,...,...,...,...


In [7]:
df = df.repartition(npartitions=5)

In [8]:
df

,Timestamp (ns),Event Type,Name,Thread,Process,Attributes
npartitions=5,,,,,,
0,float64,category[known],category[known],category[known],category[known],object
2370,...,...,...,...,...,...
...,...,...,...,...,...,...
9483,...,...,...,...,...,...
11854,...,...,...,...,...,...


In [31]:
def do_sum(df):
    return df

df.groupby(["Process", "Thread"]).apply(do_sum, meta={'Timestamp (ns)': 'float'}).compute()

2023-11-13 17:59:38,238 - distributed.shuffle - ERROR - shuffle_unpack failed during shuffle 86e7f79db4164a4b22a37f3927ae7fe0
Traceback (most recent call last):
  File "/home/rakrish/.local/lib/python3.8/site-packages/distributed/shuffle/_shuffle.py", line 78, in shuffle_unpack
    return _get_worker_extension().get_output_partition(
  File "/home/rakrish/.local/lib/python3.8/site-packages/distributed/shuffle/_worker_extension.py", line 903, in get_output_partition
    return sync(self.worker.loop, shuffle.get_output_partition, output_partition)
  File "/home/rakrish/.local/lib/python3.8/site-packages/distributed/utils.py", line 412, in sync
    raise exc.with_traceback(tb)
  File "/home/rakrish/.local/lib/python3.8/site-packages/distributed/utils.py", line 385, in f
    result = yield future
  File "/home/rakrish/.local/lib/python3.8/site-packages/tornado/gen.py", line 769, in run
    value = future.result()
  File "/home/rakrish/.local/lib/python3.8/site-packages/distributed/shuffle/

RuntimeError: shuffle_unpack failed during shuffle 86e7f79db4164a4b22a37f3927ae7fe0

In [97]:
df["Process"] = df["Process"].astype("int")
df = df.set_index("Process")
df = df.repartition(npartitions=64)

In [98]:
df

,Timestamp (ns),Event Type,Name,Thread,Attributes
npartitions=31,,,,,
0,float64,category[known],category[known],category[known],object
1,...,...,...,...,...
...,...,...,...,...,...
30,...,...,...,...,...
31,...,...,...,...,...


In [ ]:
import modin.pandas as pd
import numpy as np
df = pd.concat([pd.DataFrame(np.random.randint(0, 100, size=(2**20, 2**8))) for _ in range(40)]) # 40x2GB frames -- Working!
df.info()

To remove this warning, run the following python code before doing dataframe operations:

    import ray
    ray.init()

2023-11-12 14:35:03,060	INFO worker.py:1553 -- Started a local Ray instance.
